In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dataport
from dataport.bcm import experiment, xmatch, stack
from mlutils.data.datasets import FileTreeDataset
from pathlib import Path
import numpy as np
import h5py
from tqdm import tqdm


Connecting fabee@134.2.168.16:3306


In [3]:
restriction = dict(animal_id=23549, session=4, scan_idx=8)
restriction = [dict(animal_id=23555, session=22, scan_idx=3),  
               dict(animal_id=23555, session=23, scan_idx=1), 
               dict(animal_id=23555, session=18, scan_idx=1)]
restriction = [
    dict(animal_id=22845, session=18, scan_idx=5),  
    dict(animal_id=22845, session=18, scan_idx=8),  
]

restriction = [dict(animal_id=23555, session=30, scan_idx=6),
               dict(animal_id=23555, session=35, scan_idx=6),
               dict(animal_id=23555, session=38, scan_idx=5),
              ]

restriction = [dict(animal_id=23555 , session=5, scan_idx=12),
               dict(animal_id=23555 , session=40, scan_idx=40),
               dict(animal_id=23555 , session=39, scan_idx=24),
               dict(animal_id=23555 , session=6, scan_idx=10),
               dict(animal_id=23555 , session=42, scan_idx=9),
               dict(animal_id=23555 , session=40, scan_idx=10),
              ]
# differen collection scans
restriction = [dict(animal_id=22846, session=2, scan_idx=19),
               dict(animal_id=22846, session=2, scan_idx=21),
               dict(animal_id=23343, session=5, scan_idx=17),
              ]
scans = ['22564-2-13','22564-3-12','22846-10-16','23555-4-20','23555-5-12','23656-14-22']
scans = ['20505-6-1','20457-5-9']
# more data for NIPS paper
scans = ['23603-3-14' , '23605-1-16']
# scans for hanna
scans = ['20892-9-10', '21553-11-10', '20892-10-10']
restriction = [dict(animal_id=a, session=b, scan_idx=c) for a,b,c in map(lambda s: tuple(int(e) for e in s.split('-')), scans)]
restriction

[{'animal_id': 20892, 'session': 9, 'scan_idx': 10},
 {'animal_id': 21553, 'session': 11, 'scan_idx': 10},
 {'animal_id': 20892, 'session': 10, 'scan_idx': 10}]

In [4]:
ls cloud/static20892*.h5

cloud/static20892-10-10-preproc0.h5  cloud/static20892-6-17-preproc0.h5
cloud/static20892-3-14-preproc0.h5   cloud/static20892-6-24-preproc0.h5
cloud/static20892-4-16-preproc0.h5   cloud/static20892-9-10-preproc0.h5
cloud/static20892-5-18-preproc0.h5


In [5]:
experiment.Scan()

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,lens objective lens,brain_area short name for cortical area,aim short name for the experiment scan,filename file base name,depth manual depth measurement,scan_notes free-notes,site_number site number,software name of the software,version version,scan_ts don't edit
0,0,3,-,-,-,-,0,simulated data,0,-,-,2020-05-01 15:46:01
20457,5,9,meso,V1,2pScan,20457_5_00009,0,,0,scanimage,2017b,2018-12-13 12:08:06
20505,6,1,meso,V1,2pScan,20505_6_00001,0,,0,scanimage,2017b,2018-11-29 18:42:26
20892,9,10,meso,LM_V1,2pScan,20892_9_00010,0,,0,scanimage,2017b,2019-02-07 14:37:03
20892,10,10,meso,LM,2pScan,20892_10_00010,0,,0,scanimage,2017b,2019-02-12 14:41:04
21553,11,10,meso,LM_V1_AL_RL,2pScan,21553_11_00010,0,,0,scanimage,2017b,2019-05-24 17:14:10
22086,2,3,25x,unset,2pScan,22086_2_00003,0,,0,scanimage,2017b,2019-11-26 13:30:01
22564,2,12,meso,V1,2pScan,22564_2_00012,0,,0,scanimage,2017b,2019-09-02 15:17:37
22564,2,13,meso,V1,2pScan,22564_2_00013,0,,0,scanimage,2017b,2019-09-02 17:07:41
22564,3,8,meso,V1,2pScan,22564_3_00008,0,,0,scanimage,2017b,2019-09-03 13:12:58


In [6]:
# experiment.Scan() & restriction

template = 'cloud/static{animal_id}-{session}-{scan_idx}-preproc0'
datasets = [(template + '.h5').format(**k) 
                for k in (experiment.Scan() & restriction).fetch('KEY')]
datasets

['cloud/static20892-9-10-preproc0.h5',
 'cloud/static20892-10-10-preproc0.h5',
 'cloud/static21553-11-10-preproc0.h5']

# Count trials

In [8]:
for datafile in datasets:
    with h5py.File(datafile) as fid:
        print(datafile, fid['images'].shape)
        

cloud/static20892-9-10-preproc0.h5 (5977, 1, 36, 64)
cloud/static20892-10-10-preproc0.h5 (5992, 1, 36, 64)
cloud/static21553-11-10-preproc0.h5 (5519, 1, 36, 64)


<ipython-input-8-a15cb89ca5a2>:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(datafile) as fid:


# Export Data

In [7]:
for datafile in datasets:
    FileTreeDataset.initialize_from(datafile, overwrite=False)

/src/ml-utils/mlutils/data/utils.py:42: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5.File(filename) as fid:
Saving pupil_center: 100%|██████████| 5977/5977 [00:26<00:00, 224.84it/s]
/src/ml-utils/mlutils/data/utils.py:55: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  _savenpy(subpath / '{}.npy'.format(data_key), fid[data_key].value, overwrite)
/src/ml-utils/mlutils/data/utils.py:64: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  fid[meta_type][meta_key].value,


Not overwriting cloud/static20892-9-10-preproc0/meta/trials/tiers.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/animal_id.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/condition_hash.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/frame_image_class.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/frame_image_id.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/frame_last_flip.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/frame_pre_blank_period.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/frame_presentation_time.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/frame_trial_ts.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/scan_idx.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/session.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/trials/trial_idx.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/neurons/an

/src/ml-utils/mlutils/data/utils.py:74: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  _savenpy(outpath / 'meta/statistics' / '{}.npy'.format(name), node.value, overwrite)


Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/pupil_center/all/std.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/pupil_center/stimulus_frame/max.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/pupil_center/stimulus_frame/mean.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/pupil_center/stimulus_frame/median.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/pupil_center/stimulus_frame/min.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/pupil_center/stimulus_frame/std.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/responses/all/max.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/responses/all/mean.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/responses/all/median.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statistics/responses/all/min.npy
Not overwriting cloud/static20892-9-10-preproc0/meta/statis

Saving images:   0%|          | 16/5992 [00:00<00:38, 156.71it/s]

...

Not overwriting cloud/static20892-10-10-preproc0/meta/trials/tiers.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/animal_id.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/condition_hash.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/frame_image_class.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/frame_image_id.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/frame_last_flip.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/frame_pre_blank_period.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/frame_presentation_time.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/frame_trial_ts.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/scan_idx.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/session.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/trials/trial_idx.npy
Not overwriting cloud/static20892-10-10-preproc0/me

Saving images:   0%|          | 0/5519 [00:00<?, ?it/s]

Not overwriting cloud/static20892-10-10-preproc0/meta/statistics/responses/all/median.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/statistics/responses/all/min.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/statistics/responses/all/std.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/statistics/responses/stimulus_frame/max.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/statistics/responses/stimulus_frame/mean.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/statistics/responses/stimulus_frame/median.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/statistics/responses/stimulus_frame/min.npy
Not overwriting cloud/static20892-10-10-preproc0/meta/statistics/responses/stimulus_frame/std.npy


Saving pupil_center: 100%|██████████| 5519/5519 [00:23<00:00, 238.04it/s]


Not overwriting cloud/static21553-11-10-preproc0/meta/trials/tiers.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/animal_id.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/condition_hash.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/frame_image_class.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/frame_image_id.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/frame_last_flip.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/frame_pre_blank_period.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/frame_presentation_time.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/frame_trial_ts.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/scan_idx.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/session.npy
Not overwriting cloud/static21553-11-10-preproc0/meta/trials/trial_idx.npy
Not overwriting cloud/static21553-11-10-preproc0/me

# Link inputs and targets

In [8]:
for key in (experiment.Scan() & restriction).fetch('KEY'):
    filename = (template + '/').format(**key)
    print(filename)
    dat = FileTreeDataset(filename, 'images', 'responses')
    dat.add_link('responses', 'targets')
    dat.add_link('images', 'inputs')
    print(dat)
    
    

/usr/local/lib/python3.8/dist-packages/datajoint/connection.py:215: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


cloud/static20892-9-10-preproc0/
FileTreeDataset /notebooks/cloud/static20892-9-10-preproc0 (n=5977 items)
	images, responses
cloud/static20892-10-10-preproc0/
FileTreeDataset /notebooks/cloud/static20892-10-10-preproc0 (n=5992 items)
	images, responses
cloud/static21553-11-10-preproc0/
FileTreeDataset /notebooks/cloud/static21553-11-10-preproc0 (n=5519 items)
	images, responses


# Augment with Cell positions

In [9]:
for key in (experiment.Scan() & restriction).fetch('KEY'):
    print(key)
    filename = (template + '/').format(**key)
    dat = FileTreeDataset(filename, 'images', 'responses')
    ai, se, si, ui, x, y, z = (experiment.ScanSet.UnitInfo & key).fetch('animal_id', 'session', 'scan_idx', 'unit_id', 'um_x', 'um_y', 'um_z')
    p = np.c_[x,y,z]
    dat.add_neuron_meta('cell_motor_coordinates', ai, se, si, ui, p)

    
    

{'animal_id': 20892, 'session': 9, 'scan_idx': 10}
{'animal_id': 20892, 'session': 10, 'scan_idx': 10}
{'animal_id': 21553, 'session': 11, 'scan_idx': 10}


In [10]:
dat.neurons.cell_motor_coordinates

array([[ -248., -1315.,   250.],
       [ -267., -1338.,   250.],
       [ -218., -1329.,   250.],
       ...,
       [ -547., -1306.,   350.],
       [ -292., -1278.,   350.],
       [ -582., -1110.,   350.]])

# Augment with cell matching from StackSet

In [11]:
for key in (experiment.Scan() & restriction).proj():
    filename = (template + '/').format(**key)
    dat = FileTreeDataset(filename, 'images', 'responses')
    
    key['scan_session'] = key.pop('session')
    rel = stack.StackSet.Match() * stack.StackSet.Unit() & key
    ai, se, si, ui, match_id, mx, my, mz = rel.fetch('animal_id', 'scan_session', 'scan_idx', 'unit_id', 
                                                     'munit_id','munit_x','munit_y','munit_z')
    dat.add_neuron_meta('multi_match_id', ai, se, si, ui, match_id, fill_missing=-1)
    
    munit_coordinates = np.c_[mx, my, mz]
    dat.add_neuron_meta('multi_unit_stack_coordinates', ai, se, si, ui, munit_coordinates, fill_missing=np.nan)



Encountered 0 unmatched elements
Encountered 0 unmatched elements
Encountered 0 unmatched elements
Encountered 0 unmatched elements
Encountered 8757 unmatched elements
Encountered 8757 unmatched elements


In [ ]:
dat.neurons.multi_unit_stack_coordinates
dat.neurons.multi_match_id

# Correct the color channels 

In [ ]:
p = Path('data/static22845-18-5-preproc0/data/images/')
for filename in tqdm(p.glob('*.npy')):
    img = np.load(filename)
    if img.shape[0] == 1:
        img = np.concatenate((img, 0 * img))
        np.save(filename, img)

In [ ]:
p = Path('data/static22845-18-8-preproc0/data/images/')
for filename in tqdm(p.glob('*.npy')):
    img = np.load(filename)
    if img.shape[0] == 1:
        img = np.concatenate((0 *  img, img))
        np.save(filename, img)

# Zip Data

In [12]:
for key in (experiment.Scan() & restriction).proj():
    filename = (template + '/').format(**key)
    dat = FileTreeDataset(filename, 'images', 'responses')
    dat.zip()

23992it [06:34, 60.81it/s] 
24052it [06:17, 63.78it/s] 
22160it [05:39, 65.21it/s] 


# Clean Directories

In [13]:
import shutil

for key in (experiment.Scan() & restriction).proj():
    filename = (template + '/').format(**key)
    print(f'Removing {filename}', flush=True)
    shutil.rmtree(filename)

/usr/local/lib/python3.8/dist-packages/datajoint/connection.py:215: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


Removing cloud/static20892-9-10-preproc0/
Removing cloud/static20892-10-10-preproc0/
Removing cloud/static21553-11-10-preproc0/


# Augment with multi-cell matching (Only use this for KKLurz's data)

In [ ]:
xmatch.MatchingParameters()

In [ ]:
for key in (experiment.Scan() & 'animal_id=22564').proj():
    filename = (template + '/').format(**key)
    dat = FileTreeDataset(filename, 'images', 'responses')
    rel = xmatch.UnitMatching.Match() * xmatch.neuro_data.StaticMultiDataset.Member() & key & 'match_params=1'
    
    ai, se, si, ui, match_id = rel.fetch('animal_id', 'session', 'scan_idx', 'unit_id', 'match_id')
    dat.add_neuron_meta('multi_match_id', ai, se, si, ui, match_id, fill_missing=-1)


In [ ]:
(dat.neurons.multi_match_id < 0).sum()

In [ ]:
dat.change_log

# Lengths of datasets

In [ ]:
for key in (experiment.Scan() & 'animal_id=22564').proj():
    filename = 'data/static{animal_id}-{session}-{scan_idx}-preproc0/'.format(**key)
    dat = FileTreeDataset(filename, 'images', 'responses')
    print(len(dat))